# Análisis de Ventas con Dask

## Objetivo
Este notebook demostrará cómo manejar y analizar un conjunto de datos de ventas muy grande utilizando Dask para aprovechar el procesamiento paralelo. Compararemos el rendimiento del procesamiento con Pandas (procesamiento estándar) versus Dask.

## Análisis
Realizaremos un análisis simple sumando el total de ventas por tienda y visualizaremos los resultados. También compararemos el tiempo que toma realizar esta operación con Pandas y con Dask para evaluar el beneficio del procesamiento paralelo.

In [ ]:
import pandas as pd
import dask.dataframe as dd
import requests
import matplotlib.pyplot as plt
import time

def obtener_datos(api_url, n_iteraciones):
    # Concatenar dataframes para simular un dataset grande
    df_grande = pd.DataFrame()
    for _ in range(n_iteraciones):
        try:
            response = requests.get(api_url)
            response.raise_for_status()  # Lanzará una excepción para respuestas 4XX/5XX
            df_temp = pd.DataFrame(response.json())
            df_grande = pd.concat([df_grande, df_temp], ignore_index=True)
        except requests.RequestException as e:
            print(f"Error al realizar la solicitud: {e}")
        except ValueError:
            print("Error al decodificar JSON, verifique la URL y la estructura de la respuesta.")
    return df_grande

In [ ]:
api_url = "https://random-data-api.com/api/commerce/random_commerce?size=100"
df_grande = obtener_datos(api_url, 1)  # Ajusta '100' según necesidades y capacidades

In [ ]:
# Verificar los primeros registros para asegurarse de que los datos se han cargado correctamente
print(df_grande.head())

In [ ]:
# Convertir a Dask DataFrame
ddf = dd.from_pandas(df_grande, npartitions=10)

# Medir el tiempo de ejecución con Dask
start_time = time.time()
total_ventas_dask = ddf.groupby("department")["price"].sum().compute()
dask_time = time.time() - start_time

# Mostrar tiempo de ejecución y resultados
print(f"Tiempo de ejecución con Dask: {dask_time:.2f} segundos")
print(total_ventas_dask)

In [ ]:
# Medir el tiempo de ejecución con Pandas
start_time = time.time()
total_ventas_pandas = df_grande.groupby("department")["price"].sum()
pandas_time = time.time() - start_time

# Mostrar tiempo de ejecución y resultados
print(f"Tiempo de ejecución con Pandas: {pandas_time:.2f} segundos")
print(total_ventas_pandas)

In [ ]:
# Visualizar con Matplotlib y guardar los gráficos
fig, ax = plt.subplots()
total_ventas_dask.plot(kind='bar', ax=ax)
plt.xlabel('Departamento')
plt.ylabel('Total de Ventas')
plt.title('Ventas Totales por Departamento con Dask')
plt.savefig('ventas_dask.png')
plt.show()

fig, ax = plt.subplots()
total_ventas_pandas.plot(kind='bar', ax=ax)
plt.xlabel('Departamento')
plt.ylabel('Total de Ventas')
plt.title('Ventas Totales por Departamento con Pandas')
plt.savefig('ventas_pandas.png')
plt.show()